In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy as sp
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from nltk.corpus import stopwords
from sklearn import cross_validation
import sklearn as skl
import csv
import re
import json
import string

In [5]:
# Load Gensim features
# import gensim
# from gensim import matutils
# model = gensim.models.word2vec.Word2Vec.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

gensim_features = np.load("gensim_features.npy")
imgs = np.load("imgs.npy")
captions = np.load("captions.npy")
unique_lab = np.load("unique_lab.npy")
output = np.load("output.npy")

# Helper Functions


def n_similarity(model, ws1, ws2):
    v1 = []
    v2 = []
    bad_words = []
    for word in ws1:
        try: 
            v1.append(model[word])
        except:
            bad_words.append(word)
            continue
    for word in ws2:
        try: 
            v2.append(model[word])
        except:
            bad_words.append(word)
            continue
    return np.dot(matutils.unitvec(np.array(v1).mean(axis=0)), matutils.unitvec(np.array(v2).mean(axis=0))), bad_words


In [5]:
gensim_features = np.load("gensim_features.npy")

In [6]:
gensim_features.shape

(10000, 200)

In [5]:
#get images and labels from 10k images
labels_10k = []
labels_scores = []
for image in gensim_features:
    img_idx = np.argmax(image)
    labels_10k.append(unique_lab[np.argmax(image)])
    labels_scores.append(np.max(image))

In [36]:
# alexnet features
alex_net_feat_10k = np.load("CS5785-final-data/alexnet_feat_10k.npy")
alexnet_train_imgs = np.load("CS5785-final-data/alexnet_feat_train.npy")
alexnet_feat_test = np.load("CS5785-final-data/alexnet_feat_test.npy")
alexnet_feat_train = np.load("CS5785-final-data/alexnet_feat_train.npy")

# sift features
sift_feat_10k = np.load("CS5785-final-data/SIFTBoW_10k.npy")
sift_feat_train = np.load("CS5785-final-data/SIFTBoW_train.npy")
sift_feat_test = np.load("CS5785-final-data/SIFTBoW_test.npy")

# attributes features
def read_att(file):
    with open(file, 'r') as f:
        reader = csv.reader(f, dialect='excel', delimiter='\n')
        img = []
        fv = []
        for row in reader: 
            img.append(row[0].split()[0])
            fv.append(map(int, row[0].split()[1].split(",")))
    return img, fv

_, attributes_feat_train = read_att("CS5785-final-data/attributes_train.txt")
_, attributes_feat_test = read_att("CS5785-final-data/attributes_test.txt")
attributes_feat_train = np.array(attributes_feat_train)
attributes_feat_test = np.array(attributes_feat_test)


In [37]:
# Get attributes list
attributes_words = []
with open("CS5785-final-data/attributes_list.txt", 'r') as f:
    for row in f:
        row = row.encode('ascii','ignore')
        row = row.translate(None, string.punctuation)
        row = row.split()
        attributes_words.append(row)

In [49]:
# Get gensim attributes score
gensim_attributes_train = np.zeros((attributes_feat_train.shape[0], 200))
gensim_attributes_test = np.zeros((attributes_feat_test.shape[0], 200))
for u_idx, u in enumerate(unique_lab):
    u = u.split(' ')
    for c_idx, c in enumerate(attributes_feat_train):
        idx = np.where(c > 0)[0]
        words = np.array(attributes_words)[idx]
        all_words = []
        for w in words:
            all_words.extend(w)
        
        gensim_attributes_train[c_idx, u_idx], _ = n_similarity(model, all_words, u)
        
    for c_idx, c in enumerate(attributes_feat_test):
        idx = np.where(c > 0)[0]
        words = np.array(attributes_words)[idx]
        all_words = []
        for w in words:
            all_words.extend(w)
        
        gensim_attributes_test[c_idx, u_idx], _ = n_similarity(model, all_words, u)


In [56]:
np.sum(unique_lab[np.argmax(gensim_attributes_train, axis=1)] == output)

94

In [57]:
np.save("gensim_attributes_test.npy",gensim_attributes_test)
np.save("gensim_attributes_train.npy",gensim_attributes_train)

In [59]:
np.unique(attributes_words).shape

(102,)